In [88]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import scipy
import math
import skimage

In [2]:
data_folder = "data/"

In [70]:
img_name = "pos/per00060.ppm"
# image = cv2.imread(data_folder + img_name)
image = cv2.imread(data_folder + img_name, cv2.IMREAD_GRAYSCALE)
# cv2.imshow('Image', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [85]:
def process_image(image):
    dx = scipy.ndimage.filters.convolve1d(np.int32(image), np.array([-1, 0, 1]), 1)
    dy = scipy.ndimage.filters.convolve1d(np.int32(image), np.array([-1, 0, 1]), 0)

    magnitude = np.sqrt(dx ** 2 + dy ** 2)

    # check if image is in grayscale
    grayscale = True if len(image.shape) == 2 else False

    if not grayscale:
        max_B = np.logical_and(magnitude[:, :, 1] < magnitude[:, :, 0], magnitude[:, :, 2] < magnitude[:, :, 0])
        max_G = np.logical_and(magnitude[:, :, 0] < magnitude[:, :, 1], magnitude[:, :, 2] < magnitude[:, :, 1])
        max_R = np.logical_and(magnitude[:, :, 0] < magnitude[:, :, 2], magnitude[:, :, 1] < magnitude[:, :, 2])
        magnitude = np.array([max_R, max_G, max_B])
        magnitude = np.moveaxis(magnitude, 0, 2)

    # calculate orientation in degrees
    orientation = np.arctan2(dy, dx) * 180 / np.pi
    print(magnitude.shape, orientation.shape)


    return magnitude, orientation

def compute_histograms(magnitude, orientation, cell_size=8, num_bins=9):
    # Rozmiar obrazu
    height, width = magnitude.shape
    
    # Liczba komórek w pionie i poziomie
    cells_y = height // cell_size
    cells_x = width // cell_size
    
    # Inicjalizacja kontenera na histogramy
    histograms = np.zeros((cells_y, cells_x, num_bins))
    
    # Środek przedziałów histogramów
    bin_centers = np.linspace(0, 180, num_bins, endpoint=False) + 180 / num_bins / 2
    
    for y in range(cells_y):
        for x in range(cells_x):
            # Pobieranie wartości z odpowiedniej komórki
            mag_cell = magnitude[y*cell_size:(y+1)*cell_size, x*cell_size:(x+1)*cell_size]
            ori_cell = orientation[y*cell_size:(y+1)*cell_size, x*cell_size:(x+1)*cell_size]
            
            # Obsługa ujemnych kątów
            ori_cell[ori_cell < 0] += 180
            
            # Wyznaczanie histogramu dla komórki
            hist = np.zeros(num_bins)
            
            for i in range(cell_size):
                for j in range(cell_size):
                    mag = mag_cell[i, j]
                    ori = ori_cell[i, j]
                    
                    # Znajdowanie binu
                    bin_idx = np.searchsorted(bin_centers, ori) % num_bins
                    bin_center = bin_centers[bin_idx]
                    
                    # Wyznaczanie sąsiednich binów
                    if ori < bin_center:
                        prev_bin_idx = (bin_idx - 1) % num_bins
                    else:
                        prev_bin_idx = (bin_idx + 1) % num_bins
                    
                    # Wyznaczanie odległości od binów
                    dist_to_center = np.abs(ori - bin_center)
                    dist_to_prev = np.abs(ori - bin_centers[prev_bin_idx])
                    
                    # Interpolacja biliniowa
                    hist[bin_idx] += mag * (1 - dist_to_center / (180 / num_bins))
                    hist[prev_bin_idx] += mag * (1 - dist_to_prev / (180 / num_bins))
            
            histograms[y, x, :] = hist

          
    return histograms

def normalize_histograms(histograms, block_size=2):
    cells_y, cells_x, num_bins = histograms.shape
    e = 1e-5
    F = []

    for y in range(cells_y - 1):
        for x in range(cells_x - 1):
            H0 = histograms[y, x, :]
            H1 = histograms[y, x + 1, :]
            H2 = histograms[y + 1, x, :]
            H3 = histograms[y + 1, x + 1, :]
            H = np.concatenate((H0, H1, H2, H3))
            n = np.linalg.norm(H)
            Hn = H / np.sqrt(n**2 + e)
            F.append(Hn)

    F = np.concatenate(F)
    return F


In [90]:
def HOGpicture(w, bs): # w -histogram, bs - cell size
    bim1 = np.zeros((bs, bs))
    bim1[np.round(bs//2):np.round(bs//2)+1,:] = 1
    bim = np.zeros(bim1.shape+(9,))
    bim[:,:,0] = bim1
    for i in range(9):
        bim[:,:,i] = skimage.transform.rotate(bim1, -1*20, 'nearest')/255

    Y,X,Z = w.shape
    w[w < 0] = 0
    im = np.zeros((bs*Y, bs*X))
    for i in range(Y):
        iisl = (i)*bs
        iisu = (i+1)*bs
        for j in range(X):
            jjsl = j*bs
            jjsu = (j+1)*bs
            for k in range(9):
                im[iisl:iisu, jjsl:jjsu] += bim[:,:,k]*w[i,j,k]
    return im

In [91]:
magnitude, orientation = process_image(image)

histograms = []
if len(magnitude.shape) == 3:
    for i in range(magnitude.shape[2]):
        hist = compute_histograms(magnitude[:, :, i], orientation[:, :, i])
        plt.imshow(HOGpicture(hist, 8))
        plt.show()
        histograms.append(normalize_histograms(hist))
else:
    hist = compute_histograms(magnitude, orientation)
    plt.imshow(HOGpicture(hist, 8))
    plt.show()
    histograms.append(normalize_histograms(hist))


(128, 64) (128, 64)


/tmp/ipykernel_448475/2561276833.py:2: DeprecationWarning: Please use `convolve1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  dx = scipy.ndimage.filters.convolve1d(np.int32(image), np.array([-1, 0, 1]), 1)
/tmp/ipykernel_448475/2561276833.py:3: DeprecationWarning: Please use `convolve1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  dy = scipy.ndimage.filters.convolve1d(np.int32(image), np.array([-1, 0, 1]), 0)


ValueError: could not broadcast input array from shape (10,10) into shape (8,8)

In [75]:
for x in range(10):
    print(histograms[0][x])

-0.4052102676066403
-0.42153213774697246
0.022266785902243888
0.05327824050964546
0.06484461385154174
0.054245232971517406
0.02552632446347145
0.06198837837743398
-0.4530915692108175
-0.17488465010194948


In [ ]:
HOG_data = np.zeros([2*100, 3781], np.float32)

for i in range(100):
    IP = cv2.imread('pos/per%05d.ppm' % (i+1))
    IN = cv2.imread('neg/neg%05d.png' % (i+1))
    F = hog(IP)
    HOG_data[i,0] = 1
    HOG_data[i,1:] = F
    F = hog(IN)
    HOG_data[i+100,0] = 0
    HOG_data[i+100,1:] = F